# 🌳 1D to 2D Classification using Decision Trees

This notebook demonstrates how to classify data by learning the mapping from 1D measurements (e.g., chord lengths) to 2D shape categories using decision tree algorithms. Such tasks arise in stereology and materials analysis where observed features are lower-dimensional projections of higher-dimensional structures.


## Inferring Sphere Radius from Chord Length Distributions

This script simulates 2D cuts through randomly packed spheres and collects the resulting chord lengths. These are converted into binned histograms (feature vectors) used to identify the original sphere radius via pattern matching.


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from typing import List, Tuple, Union

# Generate non-overlapping spheres in a unit square
def generate_circles(num_circles, radius):
    positions = []
    for _ in range(num_circles):
        while True:
            x = np.random.uniform(radius, 1 - radius)
            y = np.random.uniform(radius, 1 - radius)
            if all(np.sqrt((x - px) ** 2 + (y - py) ** 2) >= 2 * radius for px, py in positions):
                positions.append((x, y))
                break
    return positions

# Collect chord lengths from all circle for one secant (one cut-line)
def calculate_chord_lengths_for_cut(a: float, b: float, 
                                  positions: List[Tuple[float, float]], 
                                  radius: float) -> List[float]:
    return [calculate_chord_length(a, b, pos, radius) for pos in positions]

# Calculate chord length for a single circle and line
def calculate_chord_length(a: float, b: float, 
                         center: Tuple[float, float], 
                         radius: float) -> float:
    x0, y0 = center
    distance = abs(a*x0 - y0 + b)/np.sqrt(a**2 + 1)
    return 2*np.sqrt(max(0, radius**2 - distance**2)) 

# Discretize a vector of chord lengths into bins
def discretize_chords(chord_lengths: List[float], 
                      num_bins: int = 15, 
                      max_length: float = 0.16) -> np.ndarray:
    bin_edges = np.linspace(0, max_length, num_bins + 1)
    hist = np.zeros(num_bins, dtype=int)

    for val in chord_lengths:
        if val == 0.0:
            continue            # this helps me to avoid collection of zeros in the first bin
        if val == max_length:
            hist[-1] += 1
            continue
        for i in range(num_bins):
            if bin_edges[i] <= val < bin_edges[i + 1]:
                hist[i] += 1
                break

    return hist

# Generate multiple discretized vectors
def generate_distribution(positions: List[Tuple[float, float]], 
                        radius: float, 
                        num_cuts: int, 
                        num_bins: int) -> List[np.ndarray]:
    distribution = []
    for _ in range(num_cuts):
        #x1, y1 = np.random.rand(2)
        #x2, y2 = np.random.rand(2)
        #while x1 == x2:
        #    x2 = np.random.rand()
        #a = (y2 - y1) / (x2 - x1)
        #b = y1 - a * x1
        a = 0
        b = np.random.uniform(0, 1)
        chords = calculate_chord_lengths_for_cut(a, b, positions, radius)
        discretized = discretize_chords(chords, num_bins)
        distribution.append(discretized)
    return distribution

# this helps to test models - the sample is in the right copy-and-paste format
def sample_from(distribution: List[np.ndarray]) -> Union[np.ndarray, None]:
    """Get first non-empty discretized vector"""
    for vec in distribution:
        if np.any(vec > 0):  # Check if any bin has counts
            return vec
    return None

## Simulating Chord Distributions for Different Sphere Densities

We simulate 2D cross-sections through sets of non-overlapping spheres (with fixed radius) at four different densities. For each case, we compute chord lengths from horizontal cuts and discretize them into histograms for further analysis.


In [13]:
# Parameters
NUM_CUTS = 1000  # Number of horizontal cuts
NUM_BINS = 10 #feature vectors
RADIUS = 0.08  # Radius of each circle

positions_21 = generate_circles(21,RADIUS)
distribution_21 = generate_distribution(positions_21, RADIUS, NUM_CUTS, NUM_BINS) #discretised vectors

positions_11 = generate_circles(11, RADIUS)
distribution_11 = generate_distribution(positions_11, RADIUS, NUM_CUTS, NUM_BINS) #discretised vectors

## Classification Task 1: Inferring Sphere Density from Chord Distributions

We aim to classify the density of spheres based on discretized chord length distributions obtained from horizontal cuts.

- **Classes**: `distribution_11` (low density) vs. `distribution_21` (high density)
- **Input**: a 15×1 feature vector (discretized chord lengths)
- **Goal**: predict which class (density level) the feature vector belongs to
- **Parameters**: 
  - `num_bins = 10` for finer discretization

This task mimics a supervised learning setting, where the model learns from labeled distributions and predicts the class of new observations.


In [15]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB


# Flatten vectors and create (X, y) pairs
def create_X_y(distribution, label):
    X = distribution
    y = np.full(len(X), label)
    return X, y

X_11, y_11 = create_X_y(distribution_11, label=0)  # low density
X_17, y_17 = create_X_y(distribution_21, label=1)  # high density

# Combine training data
X_all = np.vstack([X_11, X_17])
y_all = np.concatenate([y_11, y_17])

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2,random_state=42,stratify=y_all)



In [16]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": xgb.XGBClassifier(),
    "Naive Bayes": GaussianNB(),
    "MLP": MLPClassifier(max_iter=1000)
}



In [17]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name} Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Logistic Regression Accuracy: 0.78
SVM Accuracy: 0.83
KNN Accuracy: 0.79
Random Forest Accuracy: 0.83
XGBoost Accuracy: 0.83
Naive Bayes Accuracy: 0.70
MLP Accuracy: 0.85


In [20]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for name, model in models.items():
    accuracies = []

    for train_idx, val_idx in kf.split(X_all):
        X_train_k, X_test_k = X_all[train_idx], X_all[val_idx]
        y_train_k, y_test_k = y_all[train_idx], y_all[val_idx]

        model.fit(X_train_k, y_train_k)
        y_pred_k = model.predict(X_test_k)
        acc = accuracy_score(y_test_k, y_pred_k)
        accuracies.append(acc)

    print(f"{name} CV Accuracy: {np.mean(accuracies):.2f} ± {np.std(accuracies):.2f}")

Logistic Regression CV Accuracy: 0.78 ± 0.01
SVM CV Accuracy: 0.83 ± 0.01
KNN CV Accuracy: 0.75 ± 0.01
Random Forest CV Accuracy: 0.84 ± 0.01
XGBoost CV Accuracy: 0.84 ± 0.01
Naive Bayes CV Accuracy: 0.73 ± 0.02
MLP CV Accuracy: 0.85 ± 0.01


In [19]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    accuracies = []

    for train_idx, val_idx in skf.split(X_all, y_all):
        X_train_k, X_test_k = X_all[train_idx], X_all[val_idx]
        y_train_k, y_test_k = y_all[train_idx], y_all[val_idx]

        model.fit(X_train_k, y_train_k)
        y_pred_k = model.predict(X_test_k)
        acc = accuracy_score(y_test_k, y_pred_k)
        accuracies.append(acc)

    print(f"{name} Stratified CV Accuracy: {np.mean(accuracies):.2f} ± {np.std(accuracies):.2f}")

Logistic Regression Stratified CV Accuracy: 0.78 ± 0.03
SVM Stratified CV Accuracy: 0.83 ± 0.02
KNN Stratified CV Accuracy: 0.76 ± 0.01
Random Forest Stratified CV Accuracy: 0.83 ± 0.02
XGBoost Stratified CV Accuracy: 0.83 ± 0.02
Naive Bayes Stratified CV Accuracy: 0.72 ± 0.03
MLP Stratified CV Accuracy: 0.84 ± 0.02


In [22]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Define hyperparameter grid for XGBoost
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Initialize Stratified K-Fold (5 splits)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create XGBoost classifier (with logloss for binary classification)
xgb = XGBClassifier(eval_metric='logloss', random_state=42)

# Set up GridSearchCV for parameter tuning
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=skf,  # Using the same stratified splits
    scoring='accuracy',
    n_jobs=-1
)

# Perform grid search on the entire dataset
grid_search.fit(X_all, y_all)

# Get the best parameters and model
best_params = grid_search.best_params_
best_xgb_model = grid_search.best_estimator_

print("Best Parameters:", best_params)

# Evaluate using stratified cross-validation
cv_accuracies = []

for train_idx, val_idx in skf.split(X_all, y_all):
    X_train, X_val = X_all[train_idx], X_all[val_idx]
    y_train, y_val = y_all[train_idx], y_all[val_idx]
    
    # Train with best parameters (refit=False would skip this)
    best_xgb_model.fit(X_train, y_train)
    y_pred = best_xgb_model.predict(X_val)
    cv_accuracies.append(accuracy_score(y_val, y_pred))

# Print cross-validation results
mean_accuracy = np.mean(cv_accuracies)
std_accuracy = np.std(cv_accuracies)
print(f"Cross-Validation Accuracy: {mean_accuracy:.2f} ± {std_accuracy:.2f}")

Best Parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
Cross-Validation Accuracy: 0.84 ± 0.02


In [23]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

# Define hyperparameter grid for MLP
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],  # L2 regularization
    'learning_rate_init': [0.001, 0.01]
}

# Initialize Stratified K-Fold (consistent with XGBoost setup)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create MLP classifier with fixed parameters
mlp = MLPClassifier(
    max_iter=1000,
    early_stopping=True,  # Recommended for neural networks
    random_state=42
)

# Set up GridSearchCV
grid_mlp = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid_mlp,
    cv=skf,  # Use the same stratified splits
    scoring='accuracy',
    n_jobs=-1
)

# Perform grid search on full data (X_all, y_all)
grid_mlp.fit(X_all, y_all)

# Get best parameters and model
best_mlp_params = grid_mlp.best_params_
best_mlp_model = grid_mlp.best_estimator_

print("Best MLP Parameters:", best_mlp_params)
print("Best CV Score (from grid search):", grid_mlp.best_score_)

# Independent stratified cross-validation evaluation
cv_accuracies = []

for train_idx, val_idx in skf.split(X_all, y_all):
    X_train, X_val = X_all[train_idx], X_all[val_idx]
    y_train, y_val = y_all[train_idx], y_all[val_idx]
    
    # Train with best parameters (fresh instance)
    best_mlp_model.fit(X_train, y_train)
    y_pred = best_mlp_model.predict(X_val)
    cv_accuracies.append(accuracy_score(y_val, y_pred))

# Calculate evaluation metrics
mean_accuracy = np.mean(cv_accuracies)
std_accuracy = np.std(cv_accuracies)

print("\nIndependent Cross-Validation Results:")
print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")

Best MLP Parameters: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.01}
Best CV Score (from grid search): 0.8385

Independent Cross-Validation Results:
Accuracy: 0.8385 ± 0.0167


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

# Define hyperparameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],  # None allows full tree expansion
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2'],  # Recommended for classification
    'class_weight': [None, 'balanced']  # Handles imbalanced data
}

# Initialize Stratified K-Fold (consistent with previous models)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create Random Forest classifier
rf = RandomForestClassifier(
    random_state=42,
    n_jobs=-1  # Utilize all CPU cores
)

# Set up GridSearchCV
grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    cv=skf,
    scoring='accuracy',
    n_jobs=-1  # Parallelize grid search
)

# Full hyperparameter search
grid_rf.fit(X_all, y_all)

# Best results
best_rf_params = grid_rf.best_params_
best_rf_model = grid_rf.best_estimator_

print("Best Random Forest Parameters:", best_rf_params)
print("Grid Search CV Accuracy:", grid_rf.best_score_)

# Independent stratified cross-validation
cv_accuracies = []
feature_importances = []

for train_idx, val_idx in skf.split(X_all, y_all):
    X_train, X_val = X_all[train_idx], X_all[val_idx]
    y_train, y_val = y_all[train_idx], y_all[val_idx]
    
    # Train fresh model with best parameters
    best_rf_model.fit(X_train, y_train)
    
    # Evaluation
    y_pred = best_rf_model.predict(X_val)
    cv_accuracies.append(accuracy_score(y_val, y_pred))
    
    # Track feature importances
    feature_importances.append(best_rf_model.feature_importances_)

# Calculate metrics
mean_accuracy = np.mean(cv_accuracies)
std_accuracy = np.std(cv_accuracies)
mean_feature_importance = np.mean(feature_importances, axis=0)

print("\nIndependent Validation Results:")
print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
print("Average Feature Importances:", mean_feature_importance)

Best Random Forest Parameters: {'class_weight': None, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 100}
Grid Search CV Accuracy: 0.8414999999999999

Independent Validation Results:
Accuracy: 0.8415 ± 0.0243
Average Feature Importances: [0.00695848 0.01538156 0.03849891 0.03065802 0.05799528 0.06699828
 0.07251051 0.15153295 0.1786017  0.38086431]


In [27]:
models = {
    "Random Forest": best_rf_model,
    "XGBoost": best_xgb_model,
    "MLP": best_mlp_model
}

sample_vector_21 = None
for vec in distribution_21:
    if vec.sum() > 0:  # Skip empty cuts
        # Convert to numpy array and ensure proper numeric type
        sample_vector_21 = np.array(vec, dtype=np.float32).reshape(1, -1)
        break



print("Model Predictions:")
for name, model in models.items():
    try:
        pred = model.predict(sample_vector_21)
        print(f"{name}: {pred[0]}")
    except Exception as e:
        print(f"{name} failed: {str(e)}")

Model Predictions:
Random Forest: 1
XGBoost: 1
MLP: 1


## Regression Task 2: Inferring Sphere Radius from Chord Distributions

We now approach a **regression** problem: estimating the true **radius** of the spheres based on the shape of their chord length distributions.

- **Classes (Radii)**: choose two known radii, e.g. `r = 0.04` and `r = 0.08`
- **Input**: a 10×1 discretized feature vector from chord lengths
- **Output**: estimated radius of the sphere sample
- **Goal**: train a regression model to learn the mapping from discretized distributions to true radius values

This task demonstrates how statistical features derived from 2D cuts can be used to infer underlying 3D geometric properties.


In [30]:
######### for you to fill in

### 🔧 Lifehack: Sampling a Realistic Discretized Vector to save your time

This small code snippet is a quick way to extract a **realistic discretized vector** from a generated chord length distribution. It simulates a sphere arrangement with known radius and density, collects chord lengths from horizontal cuts, and selects the **first meaningful vector** (i.e., one that contains actual data).


In [ ]:
# Parameters
num_circles = 17
radius = 0.08
num_cuts = 100
num_bins = 15

# Generate spheres and chord lengths
positions = generate_circles(num_circles, radius)
chord_lengths_set = collect_chords_for_cuts(positions, radius, num_cuts)
distribution = distribution_chord_lengths(chord_lengths_set, num_bins)

# Extract a single discretized vector (e.g., the first non-empty one)
sample_vector = None
for vec in distribution:
    if vec.sum() > 0:  # Skip empty cuts
        sample_vector = vec
        break

# Show the result
print("Sample discretized vector:", sample_vector)

Sample discretized vector: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [106]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Ensure sample_vector is in correct shape (1, n_features)
sample_vector = np.array(sample_vector).reshape(1, -1)

# Standardize the vector using the same scaler used during training
# (Assuming you scaled your training data - if not, skip this step)
scaler = StandardScaler()
scaler.fit(X_all)  # X_all should be your original training data
sample_vector_scaled = scaler.transform(sample_vector)

# Dictionary to store predictions
predictions = {}

# Make predictions with each model
for name, model in models.items():
    try:
        pred = model.predict(sample_vector_scaled)
        proba = model.predict_proba(sample_vector_scaled) if hasattr(model, "predict_proba") else [None]
        predictions[name] = {
            'class': pred[0],
            'probability': proba[0] if proba[0] is not None else "N/A",
            'confidence': max(proba[0]) if proba[0] is not None else "N/A"
        }
    except Exception as e:
        predictions[name] = {'error': str(e)}

print("\nModel Predictions:")
print("----------------")
for name, model in models.items():
    try:
        pred = model.predict(sample_vector)
        print(f"{name}: {pred[0]}")
    except Exception as e:
        print(f"{name}: Error - {str(e)}")


Model Predictions:
----------------
Logistic Regression: 0
SVM: 0
KNN: 0
Random Forest: 0
XGBoost: 0
Naive Bayes: 0
MLP: 0
